In [1]:
import numpy as np
import matplotlib.pyplot as plt
import csv
import time

In [2]:
dataVals = np.genfromtxt(r'1zncA.txt', delimiter='', dtype=float)
avgCoords = []
indexofthresholds = []
x = 0

In [3]:
def ray_dir(E,F):
    d = E-F

In [4]:
def normal(p0,p1,p2):
    x0, y0, z0 = p0
    x1, y1, z1 = p1
    x2, y2, z2 = p2

    ux, uy, uz = u = [x1-x0, y1-y0, z1-z0] #first vector
    vx, vy, vz = v = [x2-x0, y2-y0, z2-z0] #sec vector

    u_cross_v = [uy*vz-uz*vy, uz*vx-ux*vz, ux*vy-uy*vx] #cross product

    point  = np.array(p1)
    normal = np.array(u_cross_v)
    return point,normal

def point_check(point,norm,p_plane):
    # In this instance we assume that the plane points 'up' and the arbitrary point
    # is below the plane
    dot_product = np.dot(norm,(point-p_plane))
    if dot_product <= 0: # point is below
        return -1
    elif dot_product == 0: # point is on the plane
        return 0
    else:
        return 1 # point is above plane

# This function verifies if the line segment crosses plane or not
def segment_verify(l1,l2,p1,p2,p3):
    plane_point, plane_norm = normal(p1,p2,p3)
    
    check1 = point_check(l1,plane_norm,plane_point)
    check2 = point_check(l2,plane_norm,plane_point)
    
    # If both checks return the same answer, that means line segment is not crossing the plane
    if check1 == check2:
        return False
    # If both checks return different answer that means line segment is crossing the plane
    else:
        return True

In [13]:
def intersect_line_triangle(q1, q2, p1, p2, p3):
    def signed_tetra_volume(a, b, c, d):
        return np.sign(np.dot(np.cross(b - a, c - a), d - a))

    numknots = 0
    #s1 = signed_tetra_volume(p1, p2, p3, q1)
    #s2 = signed_tetra_volume(p1, p2, p3, q2)
    val = segment_verify(q1, q2, p1, p2, p3)
    
    #The line segment crosses the plane
    if val == True:
    #if s1 != s2:
        s3 = signed_tetra_volume(p1, p2, q1, q2)
        s4 = signed_tetra_volume(p2, p3, q1, q2)
        s5 = signed_tetra_volume(p3, p1, q1, q2)
        if s3==s4 and s4==s5:
#            n = np.cross(p2 - p1, p3 - p1)
#            t = np.dot(p1 - q1, n) / np.dot(q2 - q1, n)
            numknots = numknots + 1
    # return None
    return numknots

# Strategy 1: Once threshold is reach, remove the point

In [6]:
def lineseg_dist(p, a, b):

    # normalized tangent vector
    d = np.divide(b - a, np.linalg.norm(b - a))

    # signed parallel distance components
    s = np.dot(a - p, d)
    t = np.dot(p - b, d)

    # clamped parallel distance
    h = np.maximum.reduce([s, t, 0])

    # perpendicular distance component
    c = np.cross(p - a, d)

    return np.hypot(h, np.linalg.norm(c))

# Run Code

In [14]:
dataVals = np.genfromtxt(r'1zncA.txt', delimiter='', dtype=float)
totalTime = 0
for k in range(0, 50):
    nproblem = 0
    start = time.time()

    for i in range(0, len(dataVals) - 2):

        xCoord = (dataVals[i][0] + dataVals[i + 1][0] + dataVals[i + 2][0]) / 3
        yCoord = (dataVals[i][1] + dataVals[i + 1][1] + dataVals[i + 2][1]) / 3
        zCoord = (dataVals[i][2] + dataVals[i + 1][2] + dataVals[i + 2][2]) / 3
        
        avgCoords=[xCoord, yCoord, zCoord];
        
        A = dataVals[i]
        B = dataVals[i + 1]
        C = avgCoords

        nk=0

        for j in range(0, i-2): 
            E = dataVals[j]
            F = dataVals[j + 1]
            nk += intersect_line_triangle(E, F, A, B, C)

        for j in range(i + 2, len(dataVals)-1):
            E = dataVals[j]
            F = dataVals[j + 1]
            nk += intersect_line_triangle(E, F, A, B, C)

        A = dataVals[i + 1]
        B = avgCoords
        C = dataVals[i + 2]

        for j in range(0, i-1):
            E = dataVals[j]
            F = dataVals[j + 1]
            nk += intersect_line_triangle(E, F, A, B, C)

        for j in range(i + 3, len(dataVals)-1):
            E = dataVals[j]
            F = dataVals[j + 1]
            nk += intersect_line_triangle(E, F, A, B, C)
        
        if nk==0:
            dataVals[i + 1] = avgCoords
        nproblem += nk
        
        # Check if distance is short enough
        distance = lineseg_dist(avgCoords, dataVals[i], dataVals[i+2])
        if distance < 0.01:
            indexofthresholds.append(i)
            print(i)
            
    #for i in indexofthresholds:
        #dataVals = np.delete(dataVals, i, 0)
    #indexofthresholds = []
    end = time.time()
    
    print("On iteration:", k)
    totalTime += round(end - start, 10)
    print("Time for execution of program: {}".format(round(end-start, 10)))

    print("curr possible numknot:", nproblem)
print("Total program runtime: {}".format(totalTime))

On iteration: 0
Time for execution of program: 8.527844429
curr possible numknot: 0
92
On iteration: 1
Time for execution of program: 8.1157522202
curr possible numknot: 0


KeyboardInterrupt: 